In [1]:
import pandas as pd
import numpy as np

import keras
import keras_tuner
from hp_tuning__v1_multi import LSTMHypermodel
from hp_tuning__v1_multi import TRAINING_WINDOW_SIZE, PREDICTED_WINDOW_SIZE, N_FEATURES_SEQ, N_FEATURES_STC, N_SAMPLES, BATCH_SIZE

2024-01-23 10:24:31.009223: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 10:24:31.271998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 10:24:31.272062: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 10:24:31.273854: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 10:24:31.429818: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 10:24:31.434138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
PROJECT = 'try1230'

lstm_hypermodel = LSTMHypermodel(
    training_window_size=TRAINING_WINDOW_SIZE,
    predicted_window_size=PREDICTED_WINDOW_SIZE, 
    n_features_seq=N_FEATURES_SEQ, 
    n_features_stc=N_FEATURES_STC,
    samples_per_epoch=N_SAMPLES,
    batch_size=BATCH_SIZE
)

tuner = keras_tuner.BayesianOptimization(
    hypermodel=lstm_hypermodel,
    objective=keras_tuner.Objective(name='val_root_mean_squared_error', direction='min'),
    max_trials=15,
    directory='tuning',
    project_name=PROJECT,
    overwrite=False
)

tuner.reload()

Reloading Tuner from tuning/try1230/tuner0.json


In [3]:
def retrieve_tuning_summary(tuner: keras_tuner.Tuner) -> pd.DataFrame:

    objective_name = tuner.oracle.objective.name

    results = [
        {
            **{"trial_id": trial_id}, 
            **{"objective_val": trial.metrics.get_best_value(objective_name)}, 
            **trial.hyperparameters.values
        } 
        for trial_id, trial 
        in tuner.oracle.trials.items()
    ]

    return pd.DataFrame(results).set_index('trial_id').sort_values('objective_val', ascending=True)

tuning_summary = retrieve_tuning_summary(tuner)

In [4]:
tuning_summary

,objective_val,LSTM_1_L2_recurrent,LSTM_1_dropout,LSTM_2_L2_recurrent,LSTM_2_dropout,DenseStatic_1_L2_kernel,DenseStatic_1_dropout,Dense_1_L2_kernel,Dense_1_dropout
trial_id,,,,,,,,,
07,0.083491,0.00,0.2,0.01,0.1,0.01,0.1,0.00,0.0
06,0.083577,0.00,0.2,0.01,0.0,0.01,0.0,0.00,0.0
03,0.083725,0.00,0.0,0.00,0.0,0.01,0.1,0.01,0.0
08,0.084073,0.00,0.2,0.00,0.2,0.01,0.1,0.00,0.0
02,0.084482,0.00,0.0,0.01,0.1,0.01,0.2,0.00,0.0
11,0.084680,0.01,0.1,0.00,0.0,0.01,0.0,0.01,0.0
13,0.084760,0.01,0.1,0.01,0.1,0.00,0.1,0.00,0.2
10,0.084849,0.00,0.0,0.01,0.2,0.01,0.0,0.00,0.0
04,0.085110,0.01,0.0,0.01,0.2,0.01,0.2,0.01,0.1
